In [1]:
# Import dependencies
import pandas as pd
import json
import pprint
import requests
import sys
import urllib
from config import yelp_key

In [2]:
# Set up the basics for calling the Yelp API
# https://python.gotrained.com/yelp-fusion-api-tutorial/
api_key= yelp_key
headers = {'Authorization':'Bearer %s' % api_key}
url='https://api.yelp.com/v3/businesses/search'
offset = [0,50,100,150,200,250,300,350,400,450,500,550,600,650,700,750,800,850,900]
cities = ["Kansas City MO", "St Louis MO","Springfield MO","Columbia MO","Independence MO","Lee's Summit MO","St Joseph MO","St Charles MO","St Peters MO","Blue Springs MO","Joplin MO","Florissant MO","Chesterfield MO","Wentzville MO","Jefferson City MO","Cape Girrardeau MO","Oakville MO","Wildwood MO","University City MO","Liberty MO"]
offset2 = [0,50,100]

In [3]:
# Set up empty lists to prepare for the API call
restaurant_name = []
restaurant_lat = []
restaurant_lon = []
restaurant_address = []
restaurant_state = []
loop_check = []
response_number = 0

In [4]:
# Call the API - Put N/A instead of failing
for y in cities:
    response_number = 0
    for x in offset2:
        params = {'location':[y],'price':'1','term':'fast food','limit':50,'offset':[x]}
        try:
            req=requests.get(url, params=params, headers=headers)
            response = req.json()
            for x in response["businesses"]:
                #print(response["businesses"][response_number]["name"])
                #print(response["businesses"][response_number]["coordinates"]["latitude"])
                #print(response["businesses"][response_number]["coordinates"]["longitude"])
                try:
                    restaurant_name.append(response["businesses"][response_number]["name"])
                except:
                    restaurant_name.append("N/A")

                try:
                    restaurant_lat.append(response["businesses"][response_number]["coordinates"]["latitude"])
                except:
                    restaurant_lat.append("N/A")

                try:
                    restaurant_lon.append(response["businesses"][response_number]["coordinates"]["longitude"])
                except:
                    restaurant_lon.append("N/A")

                try:
                    restaurant_address.append(response["businesses"][response_number]["location"]["display_address"])
                except:
                    restaurant_address.append("N/A")

                try:
                    restaurant_state.append(response["businesses"][response_number]["location"]["state"])
                    loop_check.append([y])
                except:
                    restaurant_state.append("N/A")
                    loop_check.append([y])

                if response_number == 49:
                    response_number = 0
                else: 
                    response_number = response_number + 1
        except:
            restaurant_name.append("N/A")
            restaurant_lat.append("N/A")
            restaurant_lon.append("N/A")
            restaurant_address.append("N/A")
            restaurant_state.append("N/A")

In [5]:
# Load into a dataframe
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html
d = {'name':restaurant_name, 'latitude':restaurant_lat,'longitude':restaurant_lon,'address':restaurant_address,'state':restaurant_state,'loopcheck':loop_check}
restaurant_df = pd.DataFrame(data=d)
restaurant_df.count()

name         2045
latitude     2045
longitude    2045
address      2045
state        2045
loopcheck    2045
dtype: int64

In [6]:
# Save to CSV
restaurant_df.to_csv("restaurant_data.csv",index=True,header=True)

In [7]:
# Filter down to unique
cleaned_restaurant_df = pd.read_csv("restaurant_data.csv")
cleaned_restaurant_df = cleaned_restaurant_df.drop_duplicates(subset= "address", keep = "first")
cleaned_restaurant_df.to_csv("cleaned_restaurant_data.csv", index = True, header = True)

In [8]:
# # Load data into Postgres
# rds_connection_string = "postgres@localhost:5433/ProjectTwoDB"
# engine = create_engine(f'postgresql://{rds_connection_string}')
# restaurant_df.to_sql(name = "restaurant_data", con = engine, if_exists='append', index = True)
# pd.read_sql_query('select * from restaurant_data', con = engine).head()